[![Buy Me a cofee](https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/coffe4.gif?raw=true ) ](https://www.buymeacoffee.com/wilzamguerrero)

In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/SDZ.png?raw=true' height="50" alt="netdata"/></center>
import os
import shutil
from IPython.utils import capture
from IPython.display import clear_output
import sys
import fileinput

SDZ_install = "/content/sample_data" #@param {type:"string"}
SDZ_Version = "V1.5" #@param ["V1.5", "V2.1"]
Download_Base_Model = "" #@param ["", "V1.5", "V1.5.inpaint", "V1.5.pix2pix", "V2.1"]
SDZ = "stable-diffusion-webui"
SDZ_webui_path = (SDZ_install+'/'+'stable-diffusion-webui')
#gitclo = "git clone --depth 1" 
gitclo = "git clone" 
os.chdir(SDZ_install)

model_init = "" #@param {type:"string"}
models_dir = "" #@param {type:"string"}
SDZ_Args = "" # @param {type:"string"}
SDZ_Dreambooth = "" #@param ["", "SDZ_New_Dreambooth", "SDZ_Old_Dreambooth"]
SDZ_Extensions = False #@param {type:"boolean"}
SDZ_Upscaler = False #@param {type:"boolean"}
SDZ_Big_Models = False #@param {type:"boolean"}
SDZ_Continue = False #@param {type:"boolean"}
SDZconfigV2="--config v2-inference-v.yaml"
share = "--share --api --cors-allow-origins=https://www.painthua.com --xformers --theme=dark --disable-safe-unpickle --enable-insecure-extension-access --no-half-vae" + " --ckpt=" + model_init + " --ckpt-dir=" + models_dir + " " + SDZ_Args

if not SDZ_Continue: 
  with capture.capture_output() as cap:
    if os.path.exists(SDZ):
      shutil.rmtree(SDZ)
    !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

    if not os.path.exists(SDZ):
      !$gitclo https://github.com/AUTOMATIC1111/stable-diffusion-webui

    %cd stable-diffusion-webui
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/config.json
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/user.css
    
    clear_output()
  print('[1;96mInstalling SDZ ✓')

if SDZ_Continue:
  with capture.capture_output() as cap:
    os.chdir(SDZ_webui_path)
    clear_output()
  print('[1;96mPreparing SDZ Continue ✓')

with capture.capture_output() as cap:
  os.chdir(SDZ_webui_path)
  ############# Requirements SDZ

  import os
  os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
  
  !pip install triton==2.0.0.dev20221202.
  !wget -N https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml
  !git clone https://github.com/bytedance/Next-ViT.git externals/Next_ViT

  %cd models/Stable-diffusion
  if not os.path.exists('/usr/local/lib/python3.8/dist-packages/open_clip'):
    !pip install xformers==0.0.16rc425
    !pip install open-clip-torch

  if Download_Base_Model == "V1.5":
    !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv1-5-fp16.ckpt -O v15.ckpt
    os.chdir(SDZ_webui_path)

  elif Download_Base_Model == "V1.5.inpaint":
    !wget https://huggingface.co/uwg/modelz_base/resolve/main/v1-5-inpaint-fp16.ckpt -O v15inp.ckpt
    os.chdir(SDZ_webui_path)

  elif Download_Base_Model == "V1.5.pix2pix":
    !wget https://huggingface.co/timbrooks/instruct-pix2pix/resolve/main/instruct-pix2pix-00-22000.safetensors -O v15pix2pix.safetensors
    os.chdir(SDZ_webui_path)
    
  elif Download_Base_Model == "V2.1":
    !wget https://huggingface.co/uwg/modelz_base/resolve/main/Zv2-1_768-fp16.ckpt -O v21.ckpt
    os.chdir(SDZ_webui_path)

  else:
    Download_Base_Model == ""
    os.chdir(SDZ_webui_path)

  if SDZ_Extensions:
    os.chdir(SDZ_webui_path)
    %cd extensions
    !git clone https://github.com/Extraltodeus/depthmap2mask.git
    !git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111.git
    !git clone https://github.com/etherealxx/sd-civitai-browser.git
    !git clone https://github.com/Vetchems/sd-model-preview.git
    !git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete.git
    !sed -i 's/dl_url = gr.Textbox(label="Download Url", interactive=False, value=None)/dl_url = gr.Textbox(label="Download Url", value=None, interactive=True, lines=1)/' $SDZ_install'stable-diffusion-webui/extensions/sd-civitai-browser/scripts/civitai-api.py'

#    if os.path.exists('sd-civitai-browser'):
#     shutil.rmtree('sd-civitai-browser')
#    !$gitclo https://github.com/etherealxx/sd-civitai-browser.git
#    if not os.path.exists('sd-civitai-browser'):
#      !$gitclo https://github.com/etherealxx/sd-civitai-browser.git
    %cd ..

  if SDZ_Upscaler:
    os.chdir(SDZ_webui_path+'/'+'models')
    if not os.path.exists('ESRGAN'):
      os.makedirs('ESRGAN')
    %cd ESRGAN
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/1x_NMKD-BrightenRedux_200k.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/1x_NMKDDetoon_97500_G.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/1x_NMKD-YandereInpaint_375000_G.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/1x_NoiseToner-Poisson-Detailed_108000_G.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/1x_NoiseToner-Uniform-Detailed_100000_G.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x_CountryRoads_377000_G.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x_fatal_Anime_500000_G.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x_Fatality_Comix_260000_G.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x_foolhardy_Remacri.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x_Nickelback_70000G.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x_NickelbackFS_72000_G.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x_NMKD-Siax_200k.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x_NMKDSuperscale_Artisoft_120000_G.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x_NMKD-Superscale-Artisoftject_210000_G.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x_NMKD-Superscale-SP_178000_G.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x_NMKD-UltraYandere_300k.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x_NMKD-UltraYandere-Lite_280k.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x_NMKD-YandereNeoXL_200k.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x_RealisticRescaler_100000_G.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4x_Valar_v1.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/4xPSNR.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/8x_NMKD-Superscale_150000_G.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/8x_NMKD-Typescale_175k.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/8xPSNR.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/16xPSNR.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/A_ESRGAN_Single.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/BSRGAN.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/BSRGANx2.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/BSRNet.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/LADDIER1_282500_G.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/lollypop.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/sudo_rife4_269.662_testV1_scale1.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/ESRGAN/WaifuGAN_v3_30000.pth
    %cd ..

    if not os.path.exists('GFPGAN'):
      os.makedirs('GFPGAN')
    %cd GFPGAN
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/GFPGAN/GFPGANv1.3.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/GFPGAN/GFPGANv1.4.pth
    %cd ..

    if not os.path.exists('SwinIR'):
      os.makedirs('SwinIR')
    %cd SwinIR
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/SwinIR/001_classicalSR_DF2K_s64w8_SwinIR-M_x2.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/SwinIR/001_classicalSR_DF2K_s64w8_SwinIR-M_x3.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/SwinIR/001_classicalSR_DF2K_s64w8_SwinIR-M_x4.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/SwinIR/001_classicalSR_DF2K_s64w8_SwinIR-M_x8.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/SwinIR/001_classicalSR_DIV2K_s48w8_SwinIR-M_x2.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/SwinIR/001_classicalSR_DIV2K_s48w8_SwinIR-M_x3.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/SwinIR/001_classicalSR_DIV2K_s48w8_SwinIR-M_x4.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/SwinIR/001_classicalSR_DIV2K_s48w8_SwinIR-M_x8.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/SwinIR/003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x2_GAN-with-dict-keys-params-and-params_ema.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/SwinIR/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN-with-dict-keys-params-and-params_ema.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/SwinIR/Swin2SR_ClassicalSR_X2_64.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/SwinIR/Swin2SR_ClassicalSR_X4_64.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/SwinIR/Swin2SR_CompressedSR_X4_48.pth
    !wget --timestamping https://huggingface.co/uwg/upscaler/resolve/main/SwinIR/Swin2SR_RealworldSR_X4_64_BSRGAN_PSNR.pth
    %cd ..
    %cd ..

  if SDZ_Dreambooth == "SDZ_New_Dreambooth":
    os.chdir(SDZ_webui_path)
    %cd extensions
    !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git
    %cd ..
    
  elif SDZ_Dreambooth == "SDZ_Old_Dreambooth":
    os.chdir(SDZ_webui_path)
    if not os.path.exists('/usr/local/lib/python3.10/site-packages/open_clip'):
      !wget -N https://github.com/korakot/kora/releases/download/v0.10/py310.sh
      !bash ./py310.sh -b -f -p /usr/local
      !python -m ipykernel install --name "py310" --user
      !conda install -c "xformers/label/dev" xformers -y
      !pip install open-clip-torch
      !pip install triton
      !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py
    
    os.chdir(SDZ_install)
    %cd stable-diffusion-webui
    !git checkout 1130d5df669911a5c67696be90bccca3ecf5f487
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/config.json
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/user_old.css -O user.css
    %cd extensions
    !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git
    %cd sd_dreambooth_extension
    !git checkout f9900ccb3ce7025d1c89334e320a1c3b255bbd07
    %cd ..
    %cd ..

  else:
    SDZ_Dreambooth == ""
    os.chdir(SDZ_webui_path)

  if SDZ_Big_Models:
    !sed -i 's@cmd_opts.lowram else \"cpu\"@cmd_opts.lowram else \"cuda\"@' $SDZ_webui_path'/modules/shared.py'
    !sed -i "s@map_location='cpu'@map_location='cuda'@" $SDZ_webui_path'/modules/extras.py'
    
    !sed -i '902s@.*@        self.logvar = self.logvar.to(self.device)@' $SDZ_webui_path'/repositories/stable-diffusion-stability-ai/ldm/models/diffusion/ddpm.py'
    !sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' $SDZ_webui_path'/webui.py'
    !sed -i "s@os.path.splitext(checkpoint_file)@os.path.splitext(checkpoint_file); map_location='cuda' if os.path.getsize(checkpoint_file) > 5500000000 else map_location@" $SDZ_webui_path'/modules/sd_models.py'
  else:
    !sed -i 's@cmd_opts.lowram else \"cuda\"@cmd_opts.lowram else \"cpu\"@' $SDZ_webui_path'/modules/shared.py'
    !sed -i "s@map_location='cuda'@map_location='cpu'@" $SDZ_webui_path'/modules/extras.py'

    clear_output()
print('[1;90mPreparing SDZ System ✓')

if SDZ_Version == "V1.5":
  !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cpu"):@' /usr/local/lib/python3.8/dist-packages/open_clip/factory.py
  !python launch.py $share
elif SDZ_Version == "V2.1":
  !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.8/dist-packages/open_clip/factory.py
  !python launch.py $share $SDZconfigV2

In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/drive.png?raw=true' height="50" alt="netdata"/></center>
from IPython.display import clear_output
from google.colab import drive
drive.mount('/content/drive')
clear_output()
print('[1;32mDrive Connected ✓')